In [1]:
from symred.symplectic_form import PauliwordOp, StabilizerOp
from symred.S3_projection import (S3_projection, 
                                  gf2_gaus_elim, 
                                  gf2_basis_for_gf2_rref, 
                                  QubitTapering, 
                                  CS_VQE, 
                                  CheatS_VQE)
from symred.build_model import build_molecule_for_projection
from symred.utils import exact_gs_energy, quasi_model
import json
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import openfermion as of
import openfermionpyscf as ofpyscf
from openfermion.circuits import ( uccsd_singlet_get_packed_amplitudes,
                                   uccsd_singlet_generator, uccsd_generator,
                                   uccsd_convert_amplitude_format)
from cirq.testing import random_unitary
from itertools import combinations, product

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open('data/molecule_data.json', 'r') as jfile:
    molecule_geometries = json.load(jfile)
with open('data/model_data.json', 'r') as jfile:
    model_data = json.load(jfile)
print(molecule_geometries.keys())

dict_keys(['H2_3-21G_SINGLET', 'H6_STO-3G_SINGLET', 'H2_6-31G_SINGLET', 'H2_6-311G_SINGLET', 'H3+_STO-3G_SINGLET', 'H3+_3-21G_SINGLET', 'HeH+_3-21G_SINGLET', 'HeH+_6-311G_SINGLET', 'H2O_STO-3G_SINGLET', 'BeH+_STO-3G_SINGLET', 'LiH_STO-3G_SINGLET', 'CH+_STO-3G_SINGLET', 'HF_STO-3G_SINGLET', 'B+_STO-3G_SINGLET', 'B_STO-3G_DOUBLET', 'N_STO-3G_QUARTET', 'OH-_STO-3G_SINGLET', 'O_STO-3G_TRIPLET', 'CH2_STO-3G_TRIPLET', 'BeH2_STO-3G_SINGLET', 'Be_STO-3G_SINGLET', 'C_STO-3G_TRIPLET', 'NH_STO-3G_SINGLET', 'Ne_STO-3G_SINGLET', 'F_STO-3G_DOUBLET', 'Li_STO-3G_DOUBLET', 'BH_STO-3G_SINGLET', 'NeH+_STO-3G_SINGLET', 'NH2+_STO-3G_SINGLET', 'BH2+_STO-3G_SINGLET', 'HCl_STO-3G_SINGLET', 'H4_STO-3G_SINGLET', 'NH3_STO-3G_SINGLET', 'F2_STO-3G_SINGLET', 'HCN_STO-3G_SINGLET', 'CH4_STO-3G_SINGLET', 'CH3OH_STO-3G_SINGLET', 'C2H6_STO-3G_SINGLET', 'CH3CN_STO-3G_SINGLET', 'CH3CHO_STO-3G_SINGLET', 'CH3CHOHCH3_STO-3G_SINGLET', 'CHONH2_STO-3G_SINGLET', 'CO2_STO-3G_SINGLET', 'O2_STO-3G_SINGLET', 'O3_STO-3G_SINGLET', 'HO

# Build the molecule with PySCF

In [3]:
# Set molecule parameters
speciesname = 'Be_STO-3G_SINGLET'
mol_data = molecule_geometries[speciesname]
if 'name' in mol_data:
    print(mol_data['name'])
    
atoms = mol_data['atoms']
coords = mol_data['coords']
basis = mol_data['basis']
multiplicity = mol_data['multiplicity']
charge = mol_data['charge']
geometry = list(zip(atoms, coords))

delete_input = True
delete_output = True
cisd=1
ccsd=1
fci =1 # wouldn't advise turning this on over 32 qubits!

# Run pyscf.
molecule_data = of.MolecularData(geometry, basis, multiplicity, charge)
calculated_molecule = ofpyscf.run_pyscf(molecule_data,
                     run_scf=1,run_mp2=1,run_cisd=cisd,run_ccsd=ccsd,run_fci=fci)

# Convert to qubits, taper and construct CS-VQE model

In [4]:
model = build_molecule_for_projection(calculated_molecule, basis_weighting='ham_coeff')

------------------------------------------------
Information concerning the full system:
------------------------------------------------
Number of qubits in full problem: 10
The Hartree-Fock state is |1111000000>
HF   energy = -14.35188048
MP2  energy = -14.37623885
CISD energy = -14.40364578
CCSD energy = -14.40365075
FCI energy  = -14.40365511
------------------------------------------------
Tapering information:
------------------------------------------------
We are able to taper 5 qubits from the Hamiltonian
The symmetry sector is:
1 ZIZIIZIZIZ +
1 IZIZIZIZIZ +
1 IIIIZZIIII +
1 IIIIIIZZII +
1 IIIIIIIIZZ
The tapered Hartree-Fock state is |11000>
------------------------------------------------
CS-VQE information:
------------------------------------------------
Noncontextual GS energy: -14.352558957524035
Symmetry generators:    
1 IIIIZ +
1 IIIZI +
1 IIZII +
-1 ZIZZZ
Clique representatives: 
-0.9997423322 IZZZZ +
-0.0226995442 IXIII
-----------------------------------------------

In [5]:
stab_indices = model.greedy_search(3)
projected = model.contextual_subspace_projection(stab_indices)
nrg = exact_gs_energy(projected.to_sparse_matrix)[0]
error = abs(nrg-model.fci_energy)
print(f'{projected.n_qubits}-qubit CS-VQE error: {error: .6f} | stabilizer indices: {stab_indices}')

1-qubit CS-VQE error:  0.031615
2-qubit CS-VQE error:  0.015027
3-qubit CS-VQE error:  0.001190
------ done ------
3-qubit CS-VQE error:  0.001190 | stabilizer indices: [0, 4]


In [6]:
taper_qubit_map = {q_pos:index for index,q_pos in enumerate(model.untapered_qubits)}

def HOMO_LUMO_hopping(N):
    assert(N<=model.ham_tap.n_qubits), 'N exceeds the full number of qubits'
    order = []
    i=1
    hop=model.HL_index
    while hop in range(model.ham.n_qubits):
        order.append(hop)
        hop+=(i)*(-1)**(i)
        i+=1
    if 0 in order:
        order+=list(range(max(order)+1, model.ham.n_qubits))
    else:
        order+=list(range(min(order)))[::-1]
    taper_order = [taper_qubit_map[i] for i in order if i not in model.tapered_qubits]
    return taper_order[:N]

def HOMO_LUMO_upper_first(N):
    upper = list(range(model.HL_index, model.ham.n_qubits))
    lower = list(range(model.HL_index))
    order = upper + lower[::-1]
    taper_order = [taper_qubit_map[i] for i in order if i not in model.tapered_qubits]
    return taper_order[:N]

def left_to_right(N):
    return list(range(N))

def right_to_left(N):
    return list(range(model.ham_tap.n_qubits - N, model.ham_tap.n_qubits)) 
    
stab_qubit_map = {q_pos:index+1 for index, q_pos in 
                     enumerate(np.argmax(model.symmetry_generators.symp_matrix!=0, axis=1) 
                               % model.ham_tap.n_qubits)}
C_qubit = list(set(range(model.ham_tap.n_qubits)).difference(stab_qubit_map.keys()))[0]
stab_qubit_map[C_qubit] = 0

def stab_ordering(N, q_ord_method):
    qubit_ordering = q_ord_method(N)
    fixed_qubits = set(range(model.ham_tap.n_qubits)).difference(qubit_ordering)
    fixed_stabs = [stab_qubit_map[i] for i in fixed_qubits]
    return fixed_stabs

if model.ham_tap.n_qubits>=16:
    N_max = 16
else:
    N_max = model.ham_tap.n_qubits
    
for N_sim in range(N_max):
    stab_indices = stab_ordering(N_sim, HOMO_LUMO_hopping)
    projected = model.contextual_subspace_projection(stab_indices)
    nrg = exact_gs_energy(projected.to_sparse_matrix)[0]
    print(f'Number of qubits:   {N_sim}\n'+
          f'Stabilizer indices: {stab_indices}\n'+
          f'Energy Error:       {abs(nrg-model.fci_energy)}')
    print()

Number of qubits:   0
Stabilizer indices: [4, 0, 3, 2, 1]
Energy Error:       0.05245293838307852

Number of qubits:   1
Stabilizer indices: [4, 3, 2, 1]
Energy Error:       0.051774631865654897

Number of qubits:   2
Stabilizer indices: [4, 2, 1]
Energy Error:       0.030817198613595664

Number of qubits:   3
Stabilizer indices: [2, 1]
Energy Error:       0.030767190908484565

Number of qubits:   4
Stabilizer indices: [1]
Energy Error:       0.014008575301271975



In [7]:
cs_vqe_errors = []
#for N_sim in range(
for comb in combinations(list(range(model.ham_tap.n_qubits)), r=2):
    comb=list(comb)
    projected = model.contextual_subspace_projection(comb)
    nrg = exact_gs_energy(projected.to_sparse_matrix)[0]
    error = abs(nrg-model.fci_energy)
    print(f'{projected.n_qubits}-qubit CS-VQE error: {error: .6f} | stabilizer indices: {comb}')
    cs_vqe_errors.append([comb, abs(nrg-model.fci_energy)])
    
print(sorted(cs_vqe_errors, key=lambda x:x[1])[0])

3-qubit CS-VQE error:  0.014927 | stabilizer indices: [0, 1]
3-qubit CS-VQE error:  0.014927 | stabilizer indices: [0, 2]
3-qubit CS-VQE error:  0.014927 | stabilizer indices: [0, 3]
3-qubit CS-VQE error:  0.001190 | stabilizer indices: [0, 4]
3-qubit CS-VQE error:  0.030767 | stabilizer indices: [1, 2]
3-qubit CS-VQE error:  0.030767 | stabilizer indices: [1, 3]
3-qubit CS-VQE error:  0.014119 | stabilizer indices: [1, 4]
3-qubit CS-VQE error:  0.030767 | stabilizer indices: [2, 3]
3-qubit CS-VQE error:  0.014119 | stabilizer indices: [2, 4]
3-qubit CS-VQE error:  0.014119 | stabilizer indices: [3, 4]
[[0, 4], 0.0011897237388165394]


# Different noncontextual forms

In [15]:
from quantumtools.Hamiltonian import QubitHamiltonian, HamiltonianGraph
from symred.S3_projection import unitary_partitioning_rotations

In [16]:
terms, coeffs = zip(*model.ham_tap.to_dictionary.items())
terms, coeffs = list(terms), list(coeffs)
graph = HamiltonianGraph(terms, coeffs)
g_dict = graph.clique_cover('AC', 'largest_first')
largest_AC_set = sorted(g_dict.items(), key=lambda x:-x[1].n_terms)[0][1]

In [22]:
anti_op = PauliwordOp(largest_AC_set.symp_matrix, largest_AC_set.coeff_vec)
print(anti_op)

-0.0466916923+0.0000000000j ZIIII +
0.0380616241-0.0000000000j XZIII +
-0.0051406428+0.0000000000j YIIIY +
0.0051406428+0.0000000000j YZYZZ +
-0.0217022449+0.0000000000j XYIIY +
-0.0217022449+0.0000000000j XXXII


In [23]:
UP_rotations = unitary_partitioning_rotations(anti_op)
print(UP_rotations)
print()
print(anti_op.recursive_rotate_by_Pword(UP_rotations).cleanup_zeros())

[('YZIII', -0.6839221871340887), ('XIIIY', -0.08513044713398724), ('XZYZZ', 0.08482363431870875), ('YYIIY', 0.3434918645830085), ('YXXII', 0.32484666743733187)]

-0.0679972693+0.0000000000j ZIIII


/home/tweaving/anaconda3/lib/python3.8/site-packages/symred/symplectic_form.py:440: UserWarning: Pword coefficient -1.00000000+0.00000000j has been set to 1
  warnings.warn(f'Pword coefficient {Pword.coeff_vec[0]: .8f} has been set to 1')
/home/tweaving/anaconda3/lib/python3.8/site-packages/symred/symplectic_form.py:440: UserWarning: Pword coefficient -1.00000000+0.00000000j has been set to 1
  warnings.warn(f'Pword coefficient {Pword.coeff_vec[0]: .8f} has been set to 1')
